In [1]:
#!pip install geopandas
#!pip install imdlib
#!pip install rasterio

In [2]:
import geopandas as gpd

In [3]:
file_name = r"C:\Users\Kanishk Goyal\OneDrive - IIT Kanpur\Desktop\Prof. Hamim Zafar\Data\Shape file for Districts India\output.shp"
gdf = gpd.read_file(file_name)

In [4]:
statename = 'Uttar Pradesh'
my = gdf[gdf.statename == statename]

In [5]:
adf = my[my.distname=='Agra']
a_shape = adf.geometry
a_shape

372    POLYGON ((77.87337 26.90196, 77.87271 26.90217...
Name: geometry, dtype: geometry

In [6]:
import imdlib as imd
import xarray as xr
import numpy as np
import pandas as pd

In [7]:
def mask_avg(xarray_,district_shape):
    import rasterio
    import rasterio.features
    import numpy as np
    lon_values = xarray_.lon.values
    lat_values = xarray_.lat.values
    lon_spacing = lon_values[1] - lon_values[0]
    lat_spacing = lat_values[1] - lat_values[0]
    transform = rasterio.transform.from_origin(lon_values[0], lat_values[-1], lon_spacing, lat_spacing)

    mask = rasterio.features.geometry_mask(district_shape, out_shape=(xarray_.lat.size, xarray_.lon.size),
                                       transform=transform, invert=True)

    masked_xarray = xarray_.where(mask)

    for var_name in masked_xarray.data_vars:
        alpha = np.nanmean(masked_xarray[var_name])
    
    return alpha

In [8]:
def avg_value(year,district_shape):
    start_yr = year
    end_yr = year
    variable_tmin = 'tmin' # other options are ('tmin'/ 'tmax')
    variable_tmax = 'tmax'
    variable_rain = 'rain'
    data_tmin = imd.get_data(variable_tmin, start_yr, end_yr, fn_format='yearwise')
    data_tmax = imd.get_data(variable_tmax, start_yr, end_yr, fn_format='yearwise')
    data_rain = imd.get_data(variable_rain, start_yr, end_yr, fn_format='yearwise')
    ds_tmin = data_tmin.get_xarray() # Converting to xarray is also required for SLICING, for a date.
    ds_tmax = data_tmax.get_xarray() # Converting to xarray is also required for SLICING, for a date.
    ds_rain = data_rain.get_xarray() # Converting to xarray is also required for SLICING, for a date.

    list_march_rain = []
    list_april_rain = []
    list_march_tmin = []
    list_april_tmin = []
    list_march_tmax = []
    list_april_tmax = []

    #print(1)

    for i in range(31):
        time_str = f'{year}-03-{i+1:02d}'
        d_tmin_march = ds_tmin.sel(time=time_str)
        d_tmax_march = ds_tmax.sel(time=time_str)
        d_rain_march = ds_rain.sel(time=time_str)
        list_march_tmin.append(d_tmin_march)
        list_march_tmax.append(d_tmax_march)
        list_march_rain.append(d_rain_march)
    
    for i, ds in enumerate(list_march_tmin):
        list_march_tmin[i] = ds.where(ds < 99, np.nan)
    for i, ds in enumerate(list_march_tmax):
        list_march_tmax[i] = ds.where(ds < 99, np.nan)  
    for i, ds in enumerate(list_march_tmin):
        list_march_rain[i] = ds.where(ds != -999, np.nan) 

    march_tmin = xr.concat(list_march_tmin, dim='dataset').mean(dim='dataset')
    march_tmax = xr.concat(list_march_tmax, dim='dataset').mean(dim='dataset')
    march_rain = xr.concat(list_march_rain, dim='dataset').mean(dim='dataset')
    
    tmin_march = mask_avg(march_tmin,district_shape)
    tmax_march = mask_avg(march_tmax,district_shape)
    rain_march = mask_avg(march_rain,district_shape)

    for j in range(30):
        time_str = f'{year}-04-{j+1:02d}'
        d_tmin_april = ds_tmin.sel(time=time_str)
        d_tmax_april = ds_tmax.sel(time=time_str)
        d_rain_april = ds_rain.sel(time=time_str)
        list_april_tmin.append(d_tmin_april)
        list_april_tmax.append(d_tmax_april)
        list_april_rain.append(d_rain_april)
    
    for i, ds in enumerate(list_april_tmin):
        list_april_tmin[i] = ds.where(ds < 99, np.nan)
    for i, ds in enumerate(list_april_tmax):
        list_april_tmax[i] = ds.where(ds < 99, np.nan)  
    for i, ds in enumerate(list_april_tmin):
        list_april_rain[i] = ds.where(ds != -999, np.nan) 

    april_tmin = xr.concat(list_april_tmin, dim='dataset').mean(dim='dataset')
    april_tmax = xr.concat(list_april_tmax, dim='dataset').mean(dim='dataset')
    april_rain = xr.concat(list_april_rain, dim='dataset').mean(dim='dataset')
    
    tmin_april = mask_avg(april_tmin,district_shape)
    tmax_april = mask_avg(april_tmax,district_shape)
    rain_april = mask_avg(april_rain,district_shape)

    #print(2)



    return tmin_march,tmax_march,rain_march, tmin_april,tmax_april,rain_april

In [9]:
data = []
for i in range(24):
    year = 1997 + i
    values = avg_value(year,a_shape)

    row_data = {'Year': year, 'tmin_march': values[0], 'tmax_march': values[1], 'rain_march': values[2],
                'tmin_april': values[3], 'tmax_april': values[4], 'rain_april': values[5]}
    
    data.append(row_data)

    print(i)
df = pd.DataFrame(data)

Downloading: mintemp for year 1997
Download Successful !!!
Downloading: maxtemp for year 1997
Download Successful !!!
Downloading: rain for year 1997
Download Successful !!!
0
Downloading: mintemp for year 1998
Download Successful !!!
Downloading: maxtemp for year 1998
Download Successful !!!
Downloading: rain for year 1998
Download Successful !!!
1
Downloading: mintemp for year 1999
Download Successful !!!
Downloading: maxtemp for year 1999
Download Successful !!!
Downloading: rain for year 1999
Download Successful !!!
2
Downloading: mintemp for year 2000
Download Successful !!!
Downloading: maxtemp for year 2000
Download Successful !!!
Downloading: rain for year 2000
Download Successful !!!
3
Downloading: mintemp for year 2001
Download Successful !!!
Downloading: maxtemp for year 2001
Download Successful !!!
Downloading: rain for year 2001
Download Successful !!!
4
Downloading: mintemp for year 2002
Download Successful !!!
Downloading: maxtemp for year 2002
Download Successful !!!
Do

In [10]:
df

,Year,tmin_march,tmax_march,rain_march,tmin_april,tmax_april,rain_april
0,1997,20.785484,36.774516,20.785484,22.654000,36.944333,22.654000
1,1998,22.379678,37.131290,22.379678,25.348333,40.190334,25.348333
2,1999,21.661936,37.346774,21.661936,24.346667,40.499667,24.346667
3,2000,20.568710,36.565806,20.568710,25.029333,40.388667,25.029333
4,2001,22.206452,36.801290,22.206452,24.635333,38.109000,24.635333
5,2002,22.008710,37.189678,22.008710,25.611333,39.733667,25.611333
6,2003,22.573226,36.664516,22.573226,25.364000,39.485334,25.364000
7,2004,21.520000,38.311290,21.520000,24.659333,39.628333,24.659333
8,2005,21.373871,36.618065,21.373871,24.136000,38.053333,24.136000
9,2006,20.296452,34.784516,20.296452,24.116000,38.682667,24.116000


In [13]:
df.to_csv(r"C:\Users\Kanishk Goyal\OneDrive - IIT Kanpur\Desktop\Prof. Hamim Zafar\Data\temp&rain\rain_temp_03_04.csv", index=False)